# Part I  - Exploration and Analysis of Loan Data from Prosper

### by F Njakai



## Table of Contents

* [Introduction](#intro)
* [Preliminary Wrangling](#prel-wrangling)
* [Univariate Exploration](#univar)
* [Bivariate Exploration](#bivar)
* [Multivariate Exploration](#multivar)
* [Summary of Findings](#summary)
* [Conclusions](#outro)

<div id="intro"></div>

## Introduction

[Prosper](https://www.prosper.com/) is a peer-to-peer lending marketplace. Borrowers apply online for a fixed-rate, fixed-term loan between USD 2,000.00 and USD 40,000.00. Individuals, like you and I, and institutions, Sequoia Capital for example, invest in said loans. Prosper handles all loan servicing on behalf of the borrowers and investors.

The data set at hand has 113,937 observations (loans, if you like) and 81 variables; detailed information on the variables can be found [here](https://docs.google.com/spreadsheets/d/1gDyi_L4UvIrLTEC6Wri5nbaMmkGmLQBk-Yx3z0XDEtI/edit#gid=0). 

>**Rubric Tip**: Your code should not generate any errors, and should use functions, loops where possible to reduce repetitive code. Prefer to use functions to reuse code statements.

> **Rubric Tip**: Document your approach and findings in markdown cells. Use comments and docstrings in code cells to document the code functionality.

>**Rubric Tip**: Markup cells should have headers and text that organize your thoughts, findings, and what you plan on investigating next.

<div id="prel-wrangling"></div>



## Preliminary Wrangling


In [5]:
#import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from random import randint

%matplotlib inline

#### Default settings for plots

Automate, as much as possible, the process of creating visualisations

Why?
* it is efficient
* vizzes will be consistent

How?
* achieve this by creating templates


In [6]:
##template no. 1

#blue colour in `seaborn` plots
default_blue = sns.color_palette('tab10')[0]

#orange colour in `seaborn` plots
default_orange = sns.color_palette('tab10')[1]


#default

In [3]:
##template no. 2

'''
simple function to create `Figure` object
using matplotlib that contains an x-lab,
y-lab and title.

"Father Figure", if you like :)

3 params, all type `str`:
x_lab, y_lab and title

Please make sure all args passed to the 
function are type `str`

return: None
'''
def create_fig(x_lab: str, y_lab: str, title: str):
    """create_fig function"""
    try:
        #fig size
        plt.figure(figsize=(10, 6.18), dpi=216)
        #x-axis name
        plt.xlabel(x_lab)
        #y-axis name 
        plt.ylabel(y_lab)
        #title
        plt.title(title)
    except ModuleNotFoundError:
        print(f'Please import matplotlib and try again')
    except:
        print(f'Failed to create template')
        raise


In [3]:
##template no. 3

'''
simple function to create `Figure` object
using matplotlib that contains an x-lab,
y-lab and title and subplots.

"Father Figure" for subplots :)

No params.

return: None
'''
def create_sub():
    """create_sub function"""
    try:
        #fig size
        fig, ax = plt.subplots(figsize=(100,61.8), dpi=2160)
    except ModuleNotFoundError:
        print(f'Please import matplotlib and try again')
    except:
        print(f'Failed to create template')
        raise


> Load in your dataset and describe its properties through the questions below. Try and motivate your exploration goals through this section.


In [4]:
#see if a df exists

'''
a simple function to see if a df exists

takes in 1 param: name of the df

Please do not pass the arg as a string

return: None
'''

def confirm_exists(df):
    """ function confirm_exists """
    if not df.empty:
        print(f'This dataframe exists')
        return
    print(f'This dataframe does not exist')
    



In [5]:
#load the data set
df = pd.read_csv('prosperLoanData.csv', sep=',')
confirm_exists(df)

This dataframe exists


In [10]:
df.shape

(113937, 81)

In [11]:
df.duplicated().value_counts()

False    113937
dtype: int64

In [8]:
df.info();

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113937 entries, 0 to 113936
Data columns (total 81 columns):
ListingKey                             113937 non-null object
ListingNumber                          113937 non-null int64
ListingCreationDate                    113937 non-null object
CreditGrade                            28953 non-null object
Term                                   113937 non-null int64
LoanStatus                             113937 non-null object
ClosedDate                             55089 non-null object
BorrowerAPR                            113912 non-null float64
BorrowerRate                           113937 non-null float64
LenderYield                            113937 non-null float64
EstimatedEffectiveYield                84853 non-null float64
EstimatedLoss                          84853 non-null float64
EstimatedReturn                        84853 non-null float64
ProsperRating (numeric)                84853 non-null float64
ProsperRating (Alpha) 

In [12]:
df.sample(randint(5, 15))

,ListingKey,ListingNumber,ListingCreationDate,CreditGrade,Term,LoanStatus,ClosedDate,BorrowerAPR,BorrowerRate,LenderYield,...,LP_ServiceFees,LP_CollectionFees,LP_GrossPrincipalLoss,LP_NetPrincipalLoss,LP_NonPrincipalRecoverypayments,PercentFunded,Recommendations,InvestmentFromFriendsCount,InvestmentFromFriendsAmount,Investors
95131,2A443364897206503365C0F,948,2006-03-01 03:08:25.490000000,HR,36,Defaulted,2006-11-08 00:00:00,0.29776,0.2900,0.2850,...,-3.18,0.00,2424.97,2424.97,0.0,1.0,0,0,0.0,32
94117,F53735261559946411A012C,526347,2011-09-08 14:42:39.037000000,NaN,36,Current,NaN,0.30532,0.2699,0.2599,...,-31.40,-13.06,0.00,0.00,0.0,1.0,0,0,0.0,14
29461,33F635700508690226D72F1,703764,2013-01-22 16:49:41.223000000,NaN,60,Chargedoff,2013-08-30 00:00:00,0.22262,0.1982,0.1882,...,-35.37,0.00,21539.69,21539.69,0.0,1.0,0,0,0.0,357
41774,15DA35369257124735F907A,553091,2012-01-21 00:51:01.963000000,NaN,60,Current,NaN,0.15784,0.1396,0.1296,...,-176.17,0.00,0.00,0.00,0.0,1.0,0,0,0.0,30
86906,B4B735819173748414A4157,805198,2013-06-11 11:12:28.510000000,NaN,60,Completed,2013-11-13 00:00:00,0.21156,0.1874,0.1774,...,-62.22,0.00,0.00,0.00,0.0,1.0,0,0,0.0,75
62677,1EFD3603289854858A47913,1246163,2014-03-08 07:51:47.843000000,NaN,60,Current,NaN,0.20321,0.1790,0.1690,...,0.00,0.00,0.00,0.00,0.0,1.0,0,0,0.0,1
82361,568C3549206755043E3DF93,593169,2012-05-26 13:23:32.223000000,NaN,36,Completed,2012-09-28 00:00:00,0.15538,0.1270,0.1170,...,-70.46,0.00,0.00,0.00,0.0,1.0,0,0,0.0,463
50369,D334350127813015687BB91,486105,2010-11-29 18:35:47.087000000,NaN,36,Completed,2014-01-17 00:00:00,0.35858,0.3220,0.3120,...,-136.16,-108.07,0.00,0.00,0.0,1.0,0,0,0.0,131
40198,8F3E35276522198506A1F80,531277,2011-10-06 11:43:05.020000000,NaN,36,Completed,2013-01-08 00:00:00,0.27467,0.2399,0.2299,...,-106.30,0.00,0.00,0.00,0.0,1.0,0,0,0.0,111
73897,4292358965888224511D914,900907,2013-09-16 18:44:56.320000000,NaN,60,Current,NaN,0.20808,0.1840,0.1740,...,-61.56,0.00,0.00,0.00,0.0,1.0,0,0,0.0,1


### Structure

#### Overall

* 113,937 observations
* 81 variables
    * 3 of type `bool`
    * 50 of type `float`
    * 11 of type `int`
    * 17 of type `str`
* more...

#### Missing and null values

* some observations in `df` have missing or null values
    * case(s) in point
        * 100,596 under variable `GroupKey`
        * 96,985 under variable `LoanFirstDefaultedCycleNumber`
    * will decide which ones to fill and/or drop on a case-by-case basis
        * Why?
            * some variables are [qualitative](https://www.statology.org/qualitative-vs-quantitative-variables/)
            * some variables apply to an observation conditionally
                * example: `GroupKey` is about groups that observations fall under; the number of said groups cannot  be reasonably expected to be 113,937
                

#### Duplicated observations

* `df` has no duplicated observations

#### Multiple values for a variable

* observations in `df` have 1 value per variable 



### Feature(s) of interest in dataset

> Your answer here!

### Feature(s) in the dataset that may help support investigation into feature(s) of interest

> Your answer here!

<div id="univar"></div>

## Univariate Exploration

> In this section, investigate distributions of individual variables. If
you see unusual points or outliers, take a deeper look to clean things up
and prepare yourself to look at relationships between variables.


> **Rubric Tip**: The project (Parts I alone) should have at least 15 visualizations distributed over univariate, bivariate, and multivariate plots to explore many relationships in the data set.  Use reasoning to justify the flow of the exploration.



>**Rubric Tip**: Use the "Question-Visualization-Observations" framework  throughout the exploration. This framework involves **asking a question from the data, creating a visualization to find answers, and then recording observations after each visualisation.** 




>**Rubric Tip**: Visualizations should depict the data appropriately so that the plots are easily interpretable. You should choose an appropriate plot type, data encodings, and formatting as needed. The formatting may include setting/adding the title, labels, legend, and comments. Also, do not overplot or incorrectly plot ordinal data.

### Discuss the distribution(s) of your variable(s) of interest. Were there any unusual points? Did you need to perform any transformations?

> Your answer here!

### Of the features you investigated, were there any unusual distributions? Did you perform any operations on the data to tidy, adjust, or change the form of the data? If so, why did you do this?

> Your answer here!

<div id="bivar"></div>

## Bivariate Exploration

> In this section, investigate relationships between pairs of variables in your
data. Make sure the variables that you cover here have been introduced in some
fashion in the previous section (univariate exploration).

### Talk about some of the relationships you observed in this part of the investigation. How did the feature(s) of interest vary with other features in the dataset?

> Your answer here!

### Did you observe any interesting relationships between the other features (not the main feature(s) of interest)?

> Your answer here!

<div id="multivar"></div>

## Multivariate Exploration

> Create plots of three or more variables to investigate your data even
further. Make sure that your investigations are justified, and follow from
your work in the previous sections.

### Talk about some of the relationships you observed in this part of the investigation. Were there features that strengthened each other in terms of looking at your feature(s) of interest?

> Your answer here!

### Were there any interesting or surprising interactions between features?

> Your answer here!

<div id="summary"></div>

## Summary of Findings

<div id="outro"></div>

## Conclusions
>You can write a summary of the main findings and reflect on the steps taken during the data exploration.



> Remove all Tips mentioned above, before you convert this notebook to PDF/HTML


> At the end of your report, make sure that you export the notebook as an
html file from the `File > Download as... > HTML or PDF` menu. Make sure you keep
track of where the exported file goes, so you can put it in the same folder
as this notebook for project submission. Also, make sure you remove all of
the quote-formatted guide notes like this one before you finish your report!

